In [1]:
%matplotlib inline
# import seaborn as sns
import numpy as np
import itertools
import csv

# import matplotlib.pyplot as plt
# from scipy import stats

In [2]:
# Experiment Design
conditions = ['tar', 'std', 'odd']
n_sbj = 30
n_trials = 100
n_std = 76
n_tar = 12
n_odd = 12
n_blocks = 8       # likely 3-4 for ECoG, max 8 for EEG

# Randomizing stimulus and response assignments
resp_opts = [0, 1]
cond_opts = [0, 1, 2]
resp_perms = list(itertools.permutations(resp_opts))
cond_perms = list(itertools.permutations(cond_opts))

# Response Randomization

In [11]:
np.random.permutation(np.arange(len(conditions)))

array([1, 0, 2])

### I'm not using this, just going in order and cycling through the permutations to make sure they're evenly covered

# Sequence Randomization

## Real Task

In [3]:
# Randomization Constraints
first_possible = 3 #10    # first trial that can be surprise
min_gap = 3    # minimum trials between surprising outcomes

assert first_possible < n_trials-(n_odd*min_gap)

In [4]:
def repeat_cnt(sequence):
    # count the number of equivalent neighbors in a sequence
    # e.g., repeat_cnt([4 2 3 3 6 10 7 7 7]) = [1 1 2 1 1 3]
    return [sum(1 for _ in group) for _, group in itertools.groupby(sequence)]

In [6]:
# Create list of trial types to randomize
cond_n = np.concatenate((np.tile(0,n_tar),np.tile(1,n_std),np.tile(2,n_odd))).astype(int)

# Create randomized trial orders
cond_blocks = np.zeros([n_blocks,n_trials])
rand_iter = 1
for b in range(n_blocks):
    cond_blocks[b,:] = np.random.permutation(cond_n)
    # Criteria:  not starting on target or oddball, no targets or oddballs within min_gap of last
    while np.where(cond_blocks[b,:]==0)[0][0] < first_possible or \
            np.where(cond_blocks[b,:]==2)[0][0] < first_possible or \
            (np.diff(np.where(cond_blocks[b,:]==0)) < min_gap).any() or \
            (np.diff(np.where(cond_blocks[b,:]==2)) < min_gap).any():
        cond_blocks[b,:] = np.random.permutation(cond_n)
        rand_iter += 1
print '# times re-randomizing = ', rand_iter

# times re-randomizing =  16695


In [8]:
for b in range(n_blocks):
    print b
    print 'first (tar, odd):', np.where(cond_blocks[b,:]==0)[0][0], np.where(cond_blocks[b,:]==2)[0][0]
    print 'min gaps (tar, odd):', np.min(np.diff(np.where(cond_blocks[b,:]==0))), np.min(np.diff(np.where(cond_blocks[b,:]==2)))

0
first (tar, odd): 12 3
min gaps (tar, odd): 4 3
1
first (tar, odd): 9 3
min gaps (tar, odd): 3 3
2
first (tar, odd): 6 18
min gaps (tar, odd): 3 3
3
first (tar, odd): 5 3
min gaps (tar, odd): 3 3
4
first (tar, odd): 11 7
min gaps (tar, odd): 4 3
5
first (tar, odd): 6 3
min gaps (tar, odd): 3 3
6
first (tar, odd): 13 6
min gaps (tar, odd): 3 3
7
first (tar, odd): 11 27
min gaps (tar, odd): 3 3


In [9]:
# Write output
with open('oddball_trial_order_csvs/block_' + str(n_trials) + '_trial_randomized_orders.csv', 'w') as write:
    writer = csv.writer(write)
    for b in range(n_blocks):
        writer.writerow(cond_blocks[b,:])

## Debug Blocks

In [11]:
# Create list of trial types to randomize
n_trials = 10
n_blocks = 8
n_std = 8
n_tar = 1
n_odd = 1
cond_n = np.concatenate((np.tile(0,n_tar),np.tile(1,n_std),np.tile(2,n_odd))).astype(int)

# Create randomized trial orders
cond_blocks = np.zeros([n_blocks,n_trials])
rand_iter = 1
for b in range(n_blocks):
    cond_blocks[b,:] = np.random.permutation(cond_n)
    # Criteria:  not starting on target or oddball, no targets or oddballs within min_gap of last
    while np.where(cond_blocks[b,:]==0)[0][0] < first_possible or \
            np.where(cond_blocks[b,:]==2)[0][0] < first_possible or \
            (np.diff(np.where(cond_blocks[b,:]==0)) < min_gap).any() or \
            (np.diff(np.where(cond_blocks[b,:]==2)) < min_gap).any():
        cond_blocks[b,:] = np.random.permutation(cond_n)
        rand_iter += 1
print '# times re-randomizing = ', rand_iter

# times re-randomizing =  11


In [12]:
# Write output
with open('oddball_trial_order_csvs/debug_' + str(n_trials) + '_trial_randomized_orders.csv', 'w') as write:
    writer = csv.writer(write)
    for b in range(n_blocks):
        writer.writerow(cond_blocks[b,:])